In [86]:
# Define project information

import sys

PROJECT_ID = "gen-lang-client-0303567819"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# if not running on colab, try to get the PROJECT_ID automatically
if "google.colab" not in sys.modules:
    import subprocess

    PROJECT_ID = subprocess.check_output(
        ["gcloud", "config", "get-value", "project"], text=True
    ).strip()

print(f"Your project ID is: {PROJECT_ID}")

Your project ID is: gen-lang-client-0303567819


Your active configuration is: [personal]


In [87]:
import sys

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [60]:
from rich import print as rich_print
from rich.markdown import Markdown as rich_Markdown
from IPython.display import Markdown, display
from vertexai.generative_models import (
    Content,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
    Image,
    Part,
)
from vertexai.language_models import TextEmbeddingModel
from vertexai.vision_models import MultiModalEmbeddingModel

In [88]:
# Multimodal models: Choose based on your performance/cost needs

multimodal_model_2_0_flash = GenerativeModel(
    "gemini-2.0-flash-001"
) # Gemini latest Gemini 2.0 Flash Model

multimodal_model_15 = GenerativeModel(
    "gemini-1.5-pro-001"
)  # works with text, code, images, video(with or without audio) and audio(mp3) with 1M input context - complex reasoning

# Multimodal models: Choose based on your performance/cost needs
multimodal_model_15_flash = GenerativeModel(
    "gemini-1.5-flash-001"
)  # works with text, code, images, video(with or without audio) and audio(mp3) with 1M input context - faster inference

/home/luizeng/Documents/fanshawe_repo/test-image-text-gemini/.venv/lib/python3.13/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [89]:
# Load text embedding model from pre-trained source
text_embedding_model = TextEmbeddingModel.from_pretrained("gemini-embedding-001")

/home/luizeng/Documents/fanshawe_repo/test-image-text-gemini/.venv/lib/python3.13/site-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
E0000 00:00:1762622587.834846   55913 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [90]:
multimodal_embedding_model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")

E0000 00:00:1762622599.768168   55913 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [91]:
from multimodal_qa_with_rag_utils import (
    get_document_metadata,
    set_global_variable,
)

set_global_variable("text_embedding_model", text_embedding_model)
set_global_variable("multimodal_embedding_model", multimodal_embedding_model)

In [65]:
# CÉLULA 78 (OPCIONAL) - 📄 EXTRAIR MAIS IMAGENS DO PDF PARA COMPARAÇÃO
# Esta célula extrai imagens do map.pdf para ter mais dados para comparação

# === EXTRAINDO IMAGENS DO PDF PARA AMPLIAR DATASET ===
#
# Esta célula verifica se há múltiplas imagens na pasta "images/".
# Se houver uma única imagem ou nenhuma, tenta extrair imagens do "map/map.pdf" (se existir).
# Caso não encontre o PDF padrão, procura outros arquivos PDF na árvore de diretórios do projeto,
# extrai imagens do primeiro encontrado e as salva em "images/".
# Ao final, imprime orientações para o usuário sobre os próximos passos.

#import os
#import fitz  # PyMuPDF
#
#def extrair_imagens_do_pdf(pdf_path, output_dir="images/", prefixo="map"):
#    """
#    Extrai imagens de um PDF e salva na pasta de imagens.
#    Args:
#        pdf_path (str): caminho do PDF de origem.
#        output_dir (str): pasta para salvar as imagens extraídas.
#        prefixo (str): prefixo opcional para nomear os arquivos de saída.
#    Returns:
#        list: caminhos das imagens extraídas.
#    """
#    print(f"🔍 Processando PDF: {pdf_path}")
#    
#    if not os.path.exists(pdf_path):
#        print(f"❌ PDF não encontrado: {pdf_path}")
#        return []
#    
#    os.makedirs(output_dir, exist_ok=True)
#    doc = fitz.open(pdf_path)
#    imagens_extraidas = []
#    
#    print(f"📊 PDF tem {len(doc)} páginas")
#    
#    for page_num in range(len(doc)):
#        page = doc[page_num]
#        images = page.get_images()
#        print(f"📄 Página {page_num + 1}: {len(images)} imagens encontradas")
#        
#        for img_index, img in enumerate(images):
#            try:
#                xref = img[0]
#                pix = fitz.Pixmap(doc, xref)
#                if pix.colorspace and pix.colorspace.n > 3:
#                    pix = fitz.Pixmap(fitz.csRGB, pix)
#                img_filename = f"{prefixo}_page_{page_num + 1}_img_{img_index + 1}.png"
#                img_path = os.path.join(output_dir, img_filename)
#                pix.save(img_path)
#                imagens_extraidas.append(img_path)
#                print(f"  ✅ Extraída: {img_filename}")
#                pix = None  # Libera memória
#            except Exception as e:
#                print(f"  ❌ Erro ao extrair imagem {img_index}: {e}")
#                continue
#    
#    doc.close()
#    print(f"\n🎉 Total de {len(imagens_extraidas)} imagens extraídas!")
#    return imagens_extraidas
#
## Conta as imagens já presentes na pasta
#current_images = len([f for f in os.listdir("images/") if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))])
#print(f"📊 Imagens atuais na pasta: {current_images}")
#
#if current_images <= 1:
#    print("🔄 Extraindo imagens do PDF para ter mais dados...")
#    # Tenta extrair do arquivo padrão
#    if os.path.exists("map/map.pdf"):
#        imagens_extraidas = extrair_imagens_do_pdf("map/map.pdf", "images/", "map")
#        if imagens_extraidas:
#            print(f"\n✅ {len(imagens_extraidas)} novas imagens adicionadas!")
#            print("🚀 Agora execute a CÉLULA 76 novamente para processar todas as imagens")
#            print("   Depois execute a CÉLULA 70 para testar similaridade com mais dados")
#        else:
#            print("❌ Nenhuma imagem foi extraída do PDF")
#    else:
#        print("❌ Arquivo map/map.pdf não encontrado")
#        
#        # Verificar outros PDFs disponíveis
#        print("\n🔍 Procurando outros PDFs...")
#        pdf_paths = []
#        for root, dirs, files in os.walk("."):
#            for file in files:
#                if file.lower().endswith('.pdf'):
#                    pdf_paths.append(os.path.join(root, file))
#        
#        if pdf_paths:
#            print("📋 PDFs encontrados:")
#            for i, pdf_path in enumerate(pdf_paths[:3], 1):  # Mostrar apenas os 3 primeiros
#                print(f"  {i}. {pdf_path}")
#                
#            # Processar o primeiro PDF encontrado
#            if pdf_paths:
#                primeiro_pdf = pdf_paths[0]
#                print(f"\n🔄 Processando: {primeiro_pdf}")
#                imagens_extraidas = extrair_imagens_do_pdf(primeiro_pdf, "images/", "doc")
#                
#                if imagens_extraidas:
#                    print(f"\n✅ {len(imagens_extraidas)} imagens extraídas de {primeiro_pdf}!")
#                    print("🚀 Execute a CÉLULA 76 novamente para processar todas as imagens")
#        else:
#            print("❌ Nenhum PDF encontrado para extrair imagens")
#            
#else:
#    print("✅ Já há múltiplas imagens na pasta")
#    print("Execute a CÉLULA 76 para processar todas e depois a CÉLULA 70 para testar similaridade")
#
## Mostrar status final
#final_images = len([f for f in os.listdir("images/") if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))])
#print(f"\n📊 STATUS FINAL: {final_images} imagens na pasta 'images/'")
#
#if final_images > 1:
#    print("🎉 Pronto para testar busca por similaridade!")
#    print("📋 PRÓXIMOS PASSOS:")
#    print("  1. Execute CÉLULA 76 (processar todas as imagens)")
#    print("  2. Execute CÉLULA 70 (busca por similaridade)")
#    print("  3. Execute CÉLULA 71 (análise contextual)")
#else:
#    print("⚠️  Ainda há apenas 1 imagem. Adicione mais imagens manualmente na pasta 'images/'")
#else:
#    print("⚠️  Ainda há apenas 1 imagem. Adicione mais imagens manualmente na pasta 'images/'")


In [92]:
# CÉLULA 75 (NOVO) - 📂 PROCESSAMENTO DIRETO DE IMAGENS DA PASTA
# Função para ler todas as imagens da pasta images/ e gerar embeddings para RAG

import os
import glob
import pandas as pd
import numpy as np
from pathlib import Path
from vertexai.generative_models import Image as GeminiImage, GenerationConfig, HarmCategory, HarmBlockThreshold
from multimodal_qa_with_rag_utils import (
    get_image_embedding_from_multimodal_embedding_model,
)

def processar_imagens_da_pasta(
    pasta_imagens="images/",
    embedding_size=512,
    gerar_descricoes=True,
    formatos_suportados=['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp']
):
    """
    Processa todas as imagens de uma pasta, gerando embeddings e descrições para RAG
    
    Args:
        pasta_imagens: Caminho da pasta com imagens
        embedding_size: Tamanho do embedding (128, 256, 512, 1408)
        gerar_descricoes: Se deve gerar descrições das imagens com Gemini
        formatos_suportados: Lista de formatos de imagem aceitos
    
    Returns:
        pd.DataFrame: DataFrame compatível com o sistema RAG existente
    """
    print(f"🔍 PROCESSANDO IMAGENS DA PASTA: {pasta_imagens}")
    print("="*60)
    
    # Verificar se a pasta existe
    if not os.path.exists(pasta_imagens):
        print(f"❌ Pasta '{pasta_imagens}' não encontrada!")
        return pd.DataFrame()
    
    # Encontrar todas as imagens na pasta
    imagens_encontradas = []
    for formato in formatos_suportados:
        pattern = os.path.join(pasta_imagens, f"*{formato}")
        imagens_encontradas.extend(glob.glob(pattern))
        pattern = os.path.join(pasta_imagens, f"*{formato.upper()}")
        imagens_encontradas.extend(glob.glob(pattern))
    
    # Remover duplicatas
    imagens_encontradas = list(set(imagens_encontradas))
    
    if not imagens_encontradas:
        print(f"❌ Nenhuma imagem encontrada na pasta '{pasta_imagens}'")
        print(f"Formatos suportados: {formatos_suportados}")
        return pd.DataFrame()
    
    print(f"📊 Encontradas {len(imagens_encontradas)} imagens:")
    for img in imagens_encontradas:
        print(f"  - {os.path.basename(img)}")
    
    # Lista para armazenar dados processados
    dados_imagens = []
    
    # Prompt para descrição das imagens
    prompt_descricao = """Analise esta imagem detalhadamente e forneça uma descrição precisa.
    Inclua:
    - O que você vê na imagem
    - Elementos principais e detalhes importantes
    - Texto visível (se houver)
    - Tipo de imagem (mapa, diagrama, foto, etc.)
    - Informações relevantes para busca e recuperação
    
    Seja específico e detalhado para facilitar buscas futuras."""
    
    print(f"\n🚀 PROCESSANDO CADA IMAGEM...")
    print("="*60)
    
    # Configure generation parameters
    generation_config = GenerationConfig(
        temperature=0.2,
        max_output_tokens=2048
    )
            
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
    }
    
    for i, caminho_imagem in enumerate(imagens_encontradas, 1):
        nome_arquivo = os.path.basename(caminho_imagem)
        print(f"\n📸 PROCESSANDO {i}/{len(imagens_encontradas)}: {nome_arquivo}")
        
        try:
            # 1. Gerar embedding da imagem
            print("  🔄 Gerando embedding...")
            image_embedding = get_image_embedding_from_multimodal_embedding_model(
                image_uri=caminho_imagem,
                embedding_size=embedding_size,
                return_array=True
            )
            print(f"  ✅ Embedding gerado: shape {image_embedding.shape}")
            
            # 2. Gerar descrição da imagem (se solicitado)
            descricao = ""
            if gerar_descricoes:
                print("  🤖 Gerando descrição com Gemini...")
                try:
                    imagem_gemini = GeminiImage.load_from_file(caminho_imagem)
                    
                    response = multimodal_model_2_0_flash.generate_content(
                        [prompt_descricao, imagem_gemini],
                        generation_config=generation_config,
                        safety_settings=safety_settings
                    )
                    
                    if response and response.text:
                        descricao = response.text
                        print(f"  ✅ Descrição gerada: {len(descricao)} caracteres")
                    else:
                        print("  ⚠️  Sem resposta do modelo")
                        descricao = f"Imagem: {nome_arquivo}"
                    
                except Exception as desc_error:
                    print(f"  ⚠️  Erro ao gerar descrição: {desc_error}")
                    descricao = f"Imagem: {nome_arquivo}"
            
            # 3. Gerar embedding da descrição (para compatibilidade com RAG)
            text_embedding = None
            if descricao:
                try:
                    from multimodal_qa_with_rag_utils import get_text_embedding_from_text_embedding_model
                    text_embedding = get_text_embedding_from_text_embedding_model(descricao)
                    print("  ✅ Text embedding da descrição gerado")
                except Exception as text_emb_error:
                    print(f"  ⚠️  Erro ao gerar text embedding: {text_emb_error}")
            
            # 4. Criar registro compatível com o sistema existente
            registro = {
                'file_name': f"pasta_images_{nome_arquivo}",  # Nome único
                'page_num': 1,  # Imagens individuais = página 1
                'img_num': i,
                'img_path': caminho_imagem,
                'img_desc': descricao,
                'mm_embedding_from_img_only': image_embedding.tolist(),  # Compatibilidade
                'text_embedding_from_image_description': text_embedding if text_embedding else None,
                'source_type': 'pasta_imagens',  # Identificar origem
                'original_filename': nome_arquivo
            }
            
            dados_imagens.append(registro)
            print(f"  ✅ Processamento concluído para {nome_arquivo}")
            
        except Exception as e:
            print(f"  ❌ Erro ao processar {nome_arquivo}: {e}")
            continue
    
    # Criar DataFrame
    if dados_imagens:
        df_imagens = pd.DataFrame(dados_imagens)
        print(f"\n🎉 PROCESSAMENTO CONCLUÍDO!")
        print(f"📊 DataFrame criado com {len(df_imagens)} imagens processadas")
        print(f"📋 Colunas: {list(df_imagens.columns)}")
        
        return df_imagens
    else:
        print(f"\n❌ Nenhuma imagem foi processada com sucesso")
        return pd.DataFrame()

print("✅ Função 'processar_imagens_da_pasta' criada com sucesso!")

✅ Função 'processar_imagens_da_pasta' criada com sucesso!


In [96]:
# CÉLULA 76 (EXECUTAR) - 🚀 PROCESSAMENTO DAS IMAGENS DA PASTA images/
# Executa o processamento de todas as imagens e cria o image_metadata_df

print("=== PROCESSAMENTO COMPLETO DA PASTA IMAGES/ ===\n")

# Executar o processamento das imagens
try:
    image_metadata_df = processar_imagens_da_pasta(
        pasta_imagens="images/",
        embedding_size=512,
        gerar_descricoes=True,  # Gerar descrições detalhadas com Gemini
        formatos_suportados=['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp']
    )
    
    if not image_metadata_df.empty:
        print(f"\n🎉 SUCESSO TOTAL!")
        print(f"📊 image_metadata_df criado com {len(image_metadata_df)} imagens")
        
        # Mostrar resumo das imagens processadas
        print(f"\n📋 RESUMO DAS IMAGENS PROCESSADAS:")
        print("="*50)
        for idx, row in image_metadata_df.iterrows():
            print(f"\n🖼️  Imagem {idx + 1}:")
            print(f"  📁 Arquivo: {row['original_filename']}")
            print(f"  📂 Caminho: {row['img_path']}")
            print(f"  📊 Embedding shape: {len(row['mm_embedding_from_img_only'])}")
            
            # Mostrar início da descrição
            desc = row['img_desc']
            if desc and len(desc) > 10:
                print(f"  📝 Descrição: {desc[:150]}{'...' if len(desc) > 150 else ''}")
        
        # Verificar compatibilidade com sistema RAG existente
        print(f"\n✅ COMPATIBILIDADE COM SISTEMA RAG:")
        colunas_necessarias = ['img_path', 'mm_embedding_from_img_only', 'img_desc', 'file_name', 'page_num']
        for col in colunas_necessarias:
            if col in image_metadata_df.columns:
                print(f"  ✅ {col}: OK")
            else:
                print(f"  ❌ {col}: FALTANDO")
        
        # Salvar para uso futuro (opcional)
        try:
            image_metadata_df.to_pickle("image_metadata_from_folder.pkl")
            print(f"\n💾 DataFrame salvo em 'image_metadata_from_folder.pkl'")
        except Exception as save_error:
            print(f"\n⚠️  Não foi possível salvar: {save_error}")
        
        print(f"\n🚀 PRÓXIMOS PASSOS:")
        print(f"1. Agora você pode executar a CÉLULA 70 (Validação)")
        print(f"2. Depois executar a CÉLULA 71 (Análise Contextual)")
        print(f"3. O sistema RAG está pronto para perguntas sobre as imagens!")
        
    else:
        print(f"\n❌ FALHA: Nenhuma imagem foi processada")
        print(f"Verifique se:")
        print(f"- A pasta 'images/' existe")
        print(f"- Há imagens válidas na pasta")
        print(f"- Os modelos estão carregados corretamente")

except Exception as e:
    print(f"❌ ERRO NO PROCESSAMENTO: {e}")
    import traceback
    traceback.print_exc()
    
    print(f"\n💡 POSSÍVEIS SOLUÇÕES:")
    print(f"- Verifique se os modelos estão carregados")
    print(f"- Verifique se a pasta 'images/' existe")
    print(f"- Execute as células de setup dos modelos primeiro")


=== PROCESSAMENTO COMPLETO DA PASTA IMAGES/ ===

🔍 PROCESSANDO IMAGENS DA PASTA: images/
📊 Encontradas 2 imagens:
  - A1.png
  - B2_room.jpeg

🚀 PROCESSANDO CADA IMAGEM...

📸 PROCESSANDO 1/2: A1.png
  🔄 Gerando embedding...
  ✅ Embedding gerado: shape (512,)
  🤖 Gerando descrição com Gemini...
  ✅ Embedding gerado: shape (512,)
  🤖 Gerando descrição com Gemini...
  ✅ Descrição gerada: 2683 caracteres
  ✅ Descrição gerada: 2683 caracteres
  ✅ Text embedding da descrição gerado
  ✅ Processamento concluído para A1.png

📸 PROCESSANDO 2/2: B2_room.jpeg
  🔄 Gerando embedding...
  ✅ Text embedding da descrição gerado
  ✅ Processamento concluído para A1.png

📸 PROCESSANDO 2/2: B2_room.jpeg
  🔄 Gerando embedding...
  ✅ Embedding gerado: shape (512,)
  🤖 Gerando descrição com Gemini...
  ✅ Embedding gerado: shape (512,)
  🤖 Gerando descrição com Gemini...
  ✅ Descrição gerada: 2478 caracteres
  ✅ Descrição gerada: 2478 caracteres
  ✅ Text embedding da descrição gerado
  ✅ Processamento concluído

In [94]:
from vertexai.generative_models import Image as GeminiImage, GenerationConfig, HarmCategory, HarmBlockThreshold
import numpy as np
from sklearn.preprocessing import normalize

def chat_about_images(question, threshold=0.7):
    """
    Chat about images using embedding similarity and Gemini model
    
    Args:
        question (str): User's question about the images
        threshold (float): Similarity threshold for image matching
    """
    try:
        # Get text embedding for the question
        question_embeddings = text_embedding_model.get_embeddings([question])
        question_embedding = np.array(question_embeddings[0].values)
        
        # Calculate similarities with all images
        similarities = []
        for _, row in image_metadata_df.iterrows():
            try:
                # Use text embedding's description embedding for comparison
                # since we can't resize the embeddings easily
                if row['text_embedding_from_image_description'] is not None:
                    compare_embedding = np.array(row['text_embedding_from_image_description'])
                    
                    # Normalize vectors for cosine similarity
                    norm_question = np.linalg.norm(question_embedding)
                    norm_compare = np.linalg.norm(compare_embedding)
                    
                    if norm_question == 0 or norm_compare == 0:
                        similarity = 0
                    else:
                        # Calculate cosine similarity
                        similarity = np.dot(question_embedding, compare_embedding) / (norm_question * norm_compare)
                    
                    similarities.append({
                        'img_path': row['img_path'],
                        'img_desc': row['img_desc'],
                        'similarity': float(similarity)  # Convert to native Python float
                    })
            except Exception as row_error:
                continue
        
        if not similarities:
            print("I could not find any relevant information to answer your question.")
            return
            
        # Sort by similarity
        similarities = sorted(similarities, key=lambda x: x['similarity'], reverse=True)
        
        # Get most relevant image
        best_match = similarities[0]
        
        if best_match['similarity'] < threshold:
            print("I could not find any relevant information to answer your question.")
            return
        
        # Generate response using Gemini
        try:
            image = GeminiImage.load_from_file(best_match['img_path'])
            
            # Configure generation parameters
            generation_config = GenerationConfig(
                temperature=0.2,
                max_output_tokens=2048
            )
            
            safety_settings = {
                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
            }
            
            prompt = f"""Question: {question}
            Image description: {best_match['img_desc']}
            
            Analyze the image and provide a clear, detailed answer to the question. If you cannot determine the answer from the image, say so."""
            
            response = multimodal_model_2_0_flash.generate_content(
                [prompt, image],
                generation_config=generation_config,
                safety_settings=safety_settings
            )
            
            if response and response.text:
                print(response.text)
            else:
                print("I apologize, but I could not generate an answer based on the available information.")
            
        except Exception as e:
            print("I apologize, but I could not generate an answer based on the available information.")
        
    except Exception as e:
        print("I apologize, but I encountered an error while processing your question.")

In [101]:
# Test different types of questions
print("Question: What appears in room B2?")
chat_about_images("What appears in room B2?")

print("\nHow many room has in the map A1?")
chat_about_images("How many room has in the map A1?")

print("\nHow can i go from 1001 room to 1033 room?")
chat_about_images("How can i go from 1004 room to 1033 room?")

Question: What appears in room B2?
Based on the image of the floor plan directory, room B2 does not exist. The directory is for "B2 Orange" on "Floor 1". The floor plan shows rooms numbered in the 2000s (e.g., 2001, 2002, 2005, etc.). Therefore, there is no room labeled "B2" on this floor plan.


How many room has in the map A1?
Based on the image of the floor plan directory, room B2 does not exist. The directory is for "B2 Orange" on "Floor 1". The floor plan shows rooms numbered in the 2000s (e.g., 2001, 2002, 2005, etc.). Therefore, there is no room labeled "B2" on this floor plan.


How many room has in the map A1?
Based on the provided floor plan of the A Building, First Floor, at 1001 Fanshawe College Blvd, the following rooms are identifiable:

1.  1001
2.  1004
3.  1005
4.  1006
5.  1007
6.  1010
7.  1012
8.  1014
9.  1016
10. 1017
11. 1018
12. 1020
13. 1033
14. 1043
15. 1046
16. 1049
17. 1051
18. 1052
19. 1054
20. 1057
21. 1058
22. 1059
23. 1063
24. 1064

Therefore, there are 